Tensorflow notebook for image classification


Resize to 64x64 to reduce processing needed, can fix this later. 

Strat:

Resize
Conv layer with pooling? 
some drop out
dense layer at the end with sigmoid output for classification
will need label vec too!

In [1]:
import tensorflow as tf
import numpy as np
import glob
from matplotlib import pyplot as plt
#from tqdm import tqdm_notebook as tqdm
import tqdm
import time

In [ ]:
def load_data(file_path):
    files = [i for i in glob.glob(path, recursive=True)]
    labels = np.array([0 if 'Picasso' in f else 1 for f in files])
    _labels = tf.one_hot(labels, depth=2, on_value=1.0, off_value=0.0)
    #print('labels!', labels)
    return files, _labels


#need labels as one hot vectors....somehow
def batch_read_and_process(files, labels, batch_size, img_h=64, img_w=64):
    images, label_index = tf.train.slice_input_producer([files, labels], shuffle=True)
    reader = tf.WholeFileReader()
    #_,img = reader.read(images) #get file contents
    img = tf.read_file(images)
    imgs = tf.image.decode_jpeg(img, channels=3) #given rgb jpegs
    imgs_resized = tf.image.resize_images(imgs, [img_w, img_h])
    print('images', imgs_resized.shape)
    #_labels = tf.one_hot(labels, depth=2)

    img_batch, label_batch = tf.train.batch([imgs_resized, label_index], batch_size=batch_size,
                                           allow_smaller_final_batch=True)
    return img_batch, label_batch
    

def model(images, training, drop_out=0.8, filters=32, kernel_size=[3,3], strides=1, activation=tf.nn.relu, padding='same'):
    with tf.variable_scope("conv1"):
        conv1_out = tf.layers.conv2d(images, filters, kernel_size, 
                                  strides=strides, activation=activation, padding=padding)
    with tf.variable_scope("conv2"):
        conv2_out = tf.layers.conv2d(conv1_out, filters, kernel_size, strides=strides,
                                    activation=activation, padding=padding)
    with tf.variable_scope("pool1"):
        pool1_out = tf.layers.max_pooling2d(conv2_out, pool_size=[2,2], strides=strides, padding='valid')
        
    with tf.variable_scope("dropout1"):
        drop1 = tf.layers.dropout(pool1_out, rate=drop_out, training=training)
        
    with tf.variable_scope("conv3"):
        conv3_out = tf.layers.conv2d(drop1, filters, kernel_size,
                                  strides=strides, activation=activation, padding=padding)
    with tf.variable_scope("conv4"):
        conv4_out = tf.layers.conv2d(conv3_out, filters, kernel_size,
                                     strides=strides, activation=activation, padding=padding)
    with tf.variable_scope("pool2"):
        pool2_out = tf.layers.max_pooling2d(conv4_out, pool_size=[2,2], strides=strides, padding='valid')
    with tf.variable_scope("dropout2"):
        drop2 = tf.layers.dropout(pool2_out, rate=drop_out, training=training)
        
    #we need to flatten our dropout tensor before we pass it to the dense layer
    with tf.variable_scope("dense"):
        drop_flat = tf.reshape(drop2, [-1, drop2.shape[1] * drop2.shape[2] * drop2.shape[3]])
        dense = tf.layers.dense(inputs=drop_flat, units=2)
    return dense
        
def loss(labels, logits):
    #print(logits.get_shape()[3])
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss
    
def accu(labels, logits):
    corr_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(corr_pred, tf.float32))
    return accuracy

def train(labels, logits, batch_size, n_epochs, learning_rate=1e-3, iters_per_epoch=None):
    print(labels)
    print(logits)
    iter_per_epoch = iters_per_epoch
    global_step = tf.Variable(0, name='global_step',trainable=False)
    
    with tf.variable_scope("loss"):
        _loss = loss(labels, logits)
        accuracy = accu(labels, logits)
    
    optimizer = tf.train.AdamOptimizer(learning_rate)
    print(_loss)
    train_op = optimizer.minimize(_loss, global_step=global_step)
    init = tf.global_variables_initializer()
    epoch_dict = {}
    with tf.Session() as sess:
        sess.run(init)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        try:
            for n in range(n_epochs):
                print('Starting epoch:  {}'.format(n))
                data_dict = {'loss':[], 'accuracy':[]}
                for _ in tqdm.trange(iter_per_epoch):
                    if not (coord.should_stop()):
                        o = sess.run([_loss, train_op, accuracy, global_step])
                        #print("Loss: {:05f}, Accuracy: {:04f}, Global Step: {:04d}".\
                        #    format(o[0], o[2], int(o[3]))) 
                        #print(labels.eval())
                        #print(logits.eval())
                        #print(tf.nn.softmax(logits).eval())
                        #print(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels).eval())
                        data_dict['loss'].append(o[0])
                        data_dict['accuracy'].append(o[2])
                epoch_dict[n] = data_dict
                epoch_loss = sum(data_dict['loss']) / len(data_dict['loss'])
                epoch_accu = sum(data_dict['accuracy']) / len(data_dict['accuracy'])
                print("Epoch: {}, Epoch Loss: {}, Epoch Accuracy: {}".format(n, epoch_loss, epoch_accu))
                

        except:
            #stage_stop.set()
            coord.request_stop()
            raise

        coord.request_stop()
        coord.join(threads)


        



        
    


    
    


In [ ]:
n_epochs = 5
batch_size = 32

path = 'artist_dataset/**/*.jpg'

_files, _labels = load_data(path)
iters_per_epoch = len(_files) // batch_size
images, labels = batch_read_and_process(_files, _labels, batch_size)
cnn = model(images, training=True)
train(labels, cnn, batch_size, n_epochs, iters_per_epoch=iters_per_epoch)





images (64, 64, 3)
Tensor("batch:1", shape=(?, 2), dtype=float32)
Tensor("dense/dense/BiasAdd:0", shape=(?, 2), dtype=float32)
Tensor("loss/Mean:0", shape=(), dtype=float32)


  0%|          | 0/62 [00:00<?, ?it/s]

Starting epoch:  0


 16%|█▌        | 10/62 [00:14<01:13,  1.41s/it]

In [ ]:
path = 'artist_dataset/**/*.jpg'

_files, _labels = load_data(path)

In [ ]:
np.array(_labels)

In [ ]:
1+1